In [42]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence

In [45]:
# %cd Downloads/NLP_data/csv_data

In [46]:
# df_train = pd.read_csv('VNCore_train_data_54.csv')
# df_test = pd.read_csv('VNCore_test_data_54.csv')

In [47]:
# %cd /content/drive/MyDrive/NLP_data

In [48]:
df_train = pd.read_csv('/content/drive/MyDrive/BERT/SA/VNCore_train_data_54.csv')
df_test = pd.read_csv('/content/drive/MyDrive/BERT/SA/VNCore_test_data_54.csv')

In [49]:
# df_train = pd.read_csv('/content/drive/MyDrive/BERT/SA/VNCore_train_data_54.csv')
# df_test = pd.read_csv('/content/drive/MyDrive/BERT/SA/VNCore_test_data_54.csv')

In [50]:
print(len(df_train))

122955


In [51]:
data = pd.concat([df_train, df_test])

In [52]:
# data_train = df_train['content_text'].map(word_tokenize).values
# data_test = df_test['content_text'].map(word_tokenize).values

In [53]:
# print(data_train[1])

In [54]:
tokenizer = text.Tokenizer(num_words=50000)
tokenizer.fit_on_texts(list(data['content']))

In [55]:
tokenized_texts = tokenizer.texts_to_sequences(df_train['content'])
X_train = sequence.pad_sequences(tokenized_texts, maxlen = 1000)

In [56]:
tokenized_texts = tokenizer.texts_to_sequences(df_test['content'])
X_test = sequence.pad_sequences(tokenized_texts, maxlen = 1000)

In [57]:
target_train = df_train['label']
target_test = df_test['label']

In [58]:
y_train = pd.get_dummies(target_train).values
y_test = pd.get_dummies(target_test).values

In [59]:
# tokenizer = text.Tokenizer(num_words=20000)
# tokenizer.fit_on_texts(list(df_train['content']))
# tokenized_texts = tokenizer.texts_to_sequences(df_train['content'])
# X_train = sequence.pad_sequences(tokenized_texts, maxlen = 100)

In [60]:
# tokenizer = text.Tokenizer(num_words=20000)
# tokenizer.fit_on_texts(list(df_test['content']))
# tokenized_texts = tokenizer.texts_to_sequences(df_test['content'])
# X_test = sequence.pad_sequences(tokenized_texts, maxlen = 100)

In [61]:
model = Sequential()

In [62]:
embedding_size = 128
model.add(Embedding(50000, embedding_size))
model.add(LSTM(128, return_sequences=True))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.5))
# model.add(Dense(50, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(54, activation='softmax'))  # use 4 because we have 4 categories

In [63]:
model.compile (loss = 'categorical_crossentropy', 
              # optimizer = 'adam', 
               optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3),
              metrics = ['accuracy'])
model.summary () 

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 128)         6400000   
                                                                 
 lstm_1 (LSTM)               (None, None, 128)         131584    
                                                                 
 global_max_pooling1d_1 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 50)                6450      
                                                                 
 dropout_3 (Dropout)         (None, 50)                0         
                                                      

In [64]:
model = tf.keras.models.load_model('/content/drive/MyDrive/Model_NLP/LSTM_54')

In [65]:
# model.fit (x = X_train, y = y_train, epochs = 10, batch_size = 32, validation_data = (X_test, y_test))

In [66]:
# from matplotlib import pyplot as plt
# history = model.fit (x = X_train, y = y_train, epochs = 10, batch_size = 32, validation_data = (X_test, y_test))
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'val'], loc='upper left')
# plt.show()

In [67]:
y_pred = model.predict(X_test)

In [68]:
print(y_test[0])

[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [69]:
print(y_pred[0])

[9.1841644e-20 7.9975458e-13 0.0000000e+00 3.0733916e-26 8.9747974e-21
 1.8861607e-24 3.6731846e-16 1.7096149e-26 1.2187427e-20 1.1301858e-24
 0.0000000e+00 4.0302402e-36 8.2991139e-13 9.9999976e-01 1.6815373e-26
 0.0000000e+00 6.5768346e-35 3.9305196e-20 8.9555979e-24 8.1691822e-14
 1.8513614e-14 8.7102648e-20 6.5698452e-18 0.0000000e+00 3.3267457e-32
 3.8970927e-12 2.2714900e-08 1.9386378e-18 0.0000000e+00 0.0000000e+00
 3.1667237e-24 1.3359945e-16 3.2092041e-30 0.0000000e+00 1.5631977e-33
 0.0000000e+00 1.0619856e-34 4.0250290e-15 2.3526965e-26 0.0000000e+00
 0.0000000e+00 1.4368117e-15 1.5014687e-16 2.6120461e-27 0.0000000e+00
 3.7330814e-22 1.2531917e-28 4.6011376e-12 2.3370798e-25 0.0000000e+00
 4.7903375e-22 1.9958892e-07 0.0000000e+00 4.9834665e-28]


In [70]:
for i in range(len(y_pred)):
  max = np.max(y_pred[i])
  print(max)
  for j in range(len(y_pred[i])):
    if y_pred[i][j] == max:
      y_pred[i][j] = 1
    else:
      y_pred[i][j] = 0

Streaming output truncated to the last 5000 lines.
1.0
0.9993586
1.0
0.9999999
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999
0.99999547
1.0
1.0
0.9999982
0.99999964
1.0
1.0
1.0
1.0
0.9999999
1.0
0.9999999
0.99999964
1.0
0.23505092
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.6737972
1.0
1.0
1.0
1.0
1.0
0.99999976
1.0
1.0
1.0
0.9999999
0.99999976
1.0
0.341443
1.0
1.0
0.52005386
1.0
0.9415935
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999
1.0
0.9999999
1.0
1.0
1.0
1.0
1.0
0.9999989
1.0
0.9999647
1.0
1.0
1.0
1.0
0.9979765
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.99999976
0.9999881
1.0
1.0
1.0
0.9999995
1.0
0.99999964
1.0
1.0
1.0
0.9999994
1.0
1.0
1.0
1.0
1.0
1.0
0.7906578
1.0
0.9999999
1.0
0.92388564
0.9999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.999997
0.36017832
0.9999999
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999
0.9999999
1.0
1.0
1.0
0.9999999
1.0
1.0
1.0
1.0
1.0
0.9999999
1.0
1.0
0.999984
0.99999964
0.99999785
1.0
0.9999999
1.0
1.0
1.0
1.0
0.9999999
1.0
1.0
0.99974936
1.0
1.0
1.0
1.0
1.0
0.9997682
1.0
0.9999808
0.9999999
0.9999999
1.0
1.0
1.

In [71]:
print(y_pred)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [72]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.77      0.85      0.81       443
           1       0.91      0.96      0.93       699
           2       0.99      0.99      0.99       626
           3       0.99      0.99      0.99       700
           4       0.99      0.99      0.99       700
           5       0.77      0.86      0.81       667
           6       0.89      0.91      0.90       700
           7       0.74      0.79      0.76       689
           8       0.80      0.57      0.66       389
           9       0.67      0.52      0.59       321
          10       0.88      0.82      0.85       644
          11       0.79      0.68      0.73       432
          12       0.83      0.63      0.72       700
          13       0.97      0.96      0.96       700
          14       0.89      0.92      0.90       699
          15       0.99      0.98      0.98       689
          16       0.81      0.88      0.84       417
          17       0.83    

In [73]:
model.save("/content/drive/MyDrive/Model_NLP/LSTM_13")

In [74]:
import keras

In [75]:
pretrained_model = keras.models.load_model("/content/drive/MyDrive/Model_NLP/LSTM_13")

In [76]:
y_pred_2 = pretrained_model.predict(X_test)

In [77]:
for i in range(len(y_pred_2)):
  max = np.max(y_pred_2[i])
  print(max)
  for j in range(len(y_pred_2[i])):
    if y_pred_2[i][j] == max:
      y_pred_2[i][j] = 1
    else:
      y_pred_2[i][j] = 0

Streaming output truncated to the last 5000 lines.
1.0
0.9993586
1.0
0.9999999
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999
0.99999547
1.0
1.0
0.9999982
0.99999964
1.0
1.0
1.0
1.0
0.9999999
1.0
0.9999999
0.99999964
1.0
0.23505092
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.6737972
1.0
1.0
1.0
1.0
1.0
0.99999976
1.0
1.0
1.0
0.9999999
0.99999976
1.0
0.341443
1.0
1.0
0.52005386
1.0
0.9415935
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999
1.0
0.9999999
1.0
1.0
1.0
1.0
1.0
0.9999989
1.0
0.9999647
1.0
1.0
1.0
1.0
0.9979765
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.99999976
0.9999881
1.0
1.0
1.0
0.9999995
1.0
0.99999964
1.0
1.0
1.0
0.9999994
1.0
1.0
1.0
1.0
1.0
1.0
0.7906578
1.0
0.9999999
1.0
0.92388564
0.9999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.999997
0.36017832
0.9999999
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999
0.9999999
1.0
1.0
1.0
0.9999999
1.0
1.0
1.0
1.0
1.0
0.9999999
1.0
1.0
0.999984
0.99999964
0.99999785
1.0
0.9999999
1.0
1.0
1.0
1.0
0.9999999
1.0
1.0
0.99974936
1.0
1.0
1.0
1.0
1.0
0.9997682
1.0
0.9999808
0.9999999
0.9999999
1.0
1.0
1.

In [78]:
print(classification_report(y_pred,y_pred_2))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       488
           1       1.00      1.00      1.00       738
           2       1.00      1.00      1.00       623
           3       1.00      1.00      1.00       698
           4       1.00      1.00      1.00       704
           5       1.00      1.00      1.00       740
           6       1.00      1.00      1.00       719
           7       1.00      1.00      1.00       741
           8       1.00      1.00      1.00       275
           9       1.00      1.00      1.00       249
          10       1.00      1.00      1.00       601
          11       1.00      1.00      1.00       371
          12       1.00      1.00      1.00       535
          13       1.00      1.00      1.00       695
          14       1.00      1.00      1.00       723
          15       1.00      1.00      1.00       682
          16       1.00      1.00      1.00       454
          17       1.00    